In [1]:
import numpy as np
import sys
sys.path.append('/home2/ebrahim/beyond-brainscore/analyze_results/figures_code')
from trained_untrained_results_funcs import find_best_layer, loop_through_datasets
import itertools
from matplotlib import pyplot as plt

In [2]:
def stack_combinations(input_dict, exclude_pairs=None, merge_sizes=None):
    
    """
    Stacks every combination of numpy arrays in the input dictionary and creates two dictionaries.
    
    Parameters:
    input_dict (dict): Dictionary where keys are strings and values are numpy arrays.
    exclude_pairs (list of tuples): List of pairs of keys that should not be stacked together.
    merge_sizes (list of tuples): List of pairs of keys whose sizes should be combined into a single value.
    
    Returns:
    tuple:
        - A dictionary with keys as combinations of the original keys and values as stacked arrays.
        - A dictionary with the same keys, but values are lists of the sizes of the concatenated arrays,
          with specified pairs having their sizes combined.
    """
    
    if exclude_pairs is None:
        exclude_pairs = []
    if merge_sizes is None:
        merge_sizes = []

    # Normalize the exclude_pairs and merge_sizes to ensure order doesn't matter
    exclude_pairs = {tuple(sorted(pair)) for pair in exclude_pairs}
    merge_sizes = {tuple(sorted(pair)) for pair in merge_sizes}
    
    output_dict = {}
    size_dict = {}
    keys = list(input_dict.keys())
    
    # Generate all combinations of keys (at least 2 keys in a combination)
    for r in range(1, len(keys) + 1):
        for combination in itertools.combinations(keys, r):
            # Check if any excluded pair exists in the combination
            skip_combination = False
            for pair in itertools.combinations(combination, 2):
                if tuple(sorted(pair)) in exclude_pairs:
                    skip_combination = True
                    break
            
            if skip_combination:
                continue
            
            # Stack the corresponding arrays
            arrays = [input_dict[key] for key in combination]
            stacked_array = np.hstack(arrays)
            
            # Create a new key by joining the original keys with '+'
            new_key = '+'.join(combination)
            output_dict[new_key] = stacked_array
            
            # Record the sizes of the concatenated arrays
            sizes = []
            merged_keys = set()
            
            for i, key1 in enumerate(combination):
                if key1 in merged_keys:
                    continue
                size = input_dict[key1].shape[1]
                
                # Check if this key should be merged with another
                for key2 in combination[i+1:]:
                    if tuple(sorted((key1, key2))) in merge_sizes:
                        size += input_dict[key2].shape[1]
                        merged_keys.add(key2)
                
                merged_keys.add(key1)
                sizes.append(size)
            
            size_dict[new_key] = sizes
    
    return output_dict, size_dict

In [11]:
best_gpt2xl_layer_dict = np.load('/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/best_layer_sigma_info/best_gpt2xl_layer.npz')
best_pereira_layers = np.load('/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/best_layer_sigma_info/best_layer_other_pereira.npz')

In [4]:
pereira_WN_model = np.load("/data/LLMs/data_processed/pereira/acts/X_word-num.npz")['layer1']
pereira_glove_sp = np.load("/data/LLMs/data_processed/pereira/acts/X_glove-sp.npz")['layer_0']
best_pos = {'384': 3.4, '243': 2.9}
pereira_all_simple_models = {'WN': pereira_WN_model, 'glove': pereira_glove_sp}

fedorenko_WP_model = np.load("/data/LLMs/data_processed/fedorenko/acts/X_pos.npz")['layer_4.7']
#fed_glove_sp = np.load("/data/LLMs/data_processed/fedorenko/acts/X_glove-sp.npz")['layer_0']
fedorenko_all_simple_models = {'WP': fedorenko_WP_model}

blank_pos_model = np.load("/data/LLMs/data_processed/blank/acts/X_pos.npz")['layer_11'] 
blank_WN_model = np.load("/data/LLMs/data_processed/blank/acts/X_WN.npz")['layer1'] 
#blank_glove_sp = np.load("/data/LLMs/data_processed/blank/acts/X_glove-sp.npz")['layer_0']
blank_all_simple_models = {'pos': blank_pos_model, 'WN': blank_WN_model}

all_simple_models = {'pereira': pereira_all_simple_models, 'fedorenko': fedorenko_all_simple_models, 'blank': blank_all_simple_models}

In [35]:
for dataset, fe, exp, subjects, network in loop_through_datasets(['pereira', 'fedorenko', 'blank'], ['', '-mp', '-sp']):

    gpt2xl_acts = np.load(f"/data/LLMs/data_processed/{dataset}/acts/X_gpt2-xl{fe}.npz")
    
    gpt2xl_best_layer = best_gpt2xl_layer_dict[f'{dataset}{exp}_out_of_sample_r2_contig{fe}']

    all_simple_models[dataset]['gpt2xl'] = gpt2xl_acts[f"layer_{gpt2xl_best_layer}"]
    
    
    if dataset == 'pereira':
        exp = exp.strip('_')
        position_pereira = np.load("/data/LLMs/data_processed/pereira/acts/X_position.npz")[f'layer_{best_pos[exp]}']
        all_simple_models[dataset]['pos'] = position_pereira
        
        gpt2xl_acts_decontext = np.load(f"/data/LLMs/data_processed/{dataset}/acts/X_gpt2-xl-decontext{fe}.npz")
        
        gpt2_decontext_best_layer = best_gpt2xl_layer_dict[f'{dataset}_{exp}_pearson_r_contig{fe}']
        print(gpt2_decontext_best_layer)
        
        gpt2_decontext_p = {'layer1': np.hstack((gpt2xl_acts_decontext[f"layer_{gpt2_decontext_best_layer}"], position_pereira))}
        
        np.savez(f'/data/LLMs/data_processed/{dataset}/acts/X_gpt2xl-decontext_position_{exp}{fe}_pearsonr', **gpt2_decontext_p)
        
        
    all_simple_models_dict, feature_list_dict = stack_combinations(all_simple_models[dataset], merge_sizes=[('pos', 'WN')])
    
    np.savez(f'/data/LLMs/data_processed/{dataset}/acts/X_trained-var-par{exp}{fe}', **all_simple_models_dict)
    np.savez(f'/data/LLMs/data_processed/{dataset}/acts/f-list_trained-var-par{exp}{fe}', **feature_list_dict)
        
    

15
21
16
21
16
16


In [17]:
best_pereira_layers

NpzFile '/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/best_layer_sigma_info/best_layer_other_pereira.npz' with keys: PWR_pearson_r_243, GPT2XL-decontext_pearson_r_243_, PWR_out_of_sample_r2_243, GPT2XL-decontext_out_of_sample_r2_243_, PWR_pearson_r_384...

In [ ]:
gpt2_xl_decontext_best_layers = {'-lt': 24, '-mp': }